<a href="https://colab.research.google.com/github/AmeerHamza-345/Q2/blob/main/Project_2_LangChain_RAG_Project(Educational_Quiz_Generator).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!pip install --q --upgrade langchain langchain_community pinecone-client google-generativeai openai tqdm
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.7 MB/s eta 0:00:00


In [73]:
import langchain_google_genai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [74]:
# from google.colab import userdata
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# PINECONE_API_KEY= userdata.get('PINECONE_API_KEY')
# if GOOGLE_API_KEY and PINECONE_API_KEY:
#   print("API keys is loaded")
# else:
#   print("API keys is not loaded")
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [75]:
model_2_0:ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
  model = "gemini-2.0-flash-exp",
  api_key = GOOGLE_API_KEY
)

In [76]:
response = model_2_0.invoke("hello")
print(response.content)

Hi there! How can I help you today?



In [77]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/content/educational_quiz_generator.csv")

dataset = loader.load()

print(dataset)

[Document(metadata={'source': '/content/educational_quiz_generator.csv', 'row': 0}, page_content='Topic: Mathematics\nQuestion: Question 1 on Mathematics'), Document(metadata={'source': '/content/educational_quiz_generator.csv', 'row': 1}, page_content='Topic: Science\nQuestion: Question 2 on Science'), Document(metadata={'source': '/content/educational_quiz_generator.csv', 'row': 2}, page_content='Topic: History\nQuestion: Question 3 on History'), Document(metadata={'source': '/content/educational_quiz_generator.csv', 'row': 3}, page_content='Topic: Geography\nQuestion: Question 4 on Geography'), Document(metadata={'source': '/content/educational_quiz_generator.csv', 'row': 4}, page_content='Topic: Literature\nQuestion: Question 5 on Literature'), Document(metadata={'source': '/content/educational_quiz_generator.csv', 'row': 5}, page_content='Topic: Mathematics\nQuestion: Question 6 on Mathematics'), Document(metadata={'source': '/content/educational_quiz_generator.csv', 'row': 6}, pa

In [78]:
from langchain.text_splitter import CharacterTextSplitter # Import the CharacterTextSplitter class
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(dataset)

In [79]:
print(chunks[0])
print(len(chunks))

page_content='Topic: Mathematics
Question: Question 1 on Mathematics' metadata={'source': '/content/educational_quiz_generator.csv', 'row': 0}
1000


In [80]:
chunks[3]

Document(metadata={'source': '/content/educational_quiz_generator.csv', 'row': 3}, page_content='Topic: Geography\nQuestion: Question 4 on Geography')

In [81]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
# os.environ['PINECONE_ENV'] = userdata.get('pinecone_env')
api_key = os.environ.get('PINECONE_API_KEY')

In [82]:
PINECONE_API_KEY=PINECONE_API_KEY
PINECONE_ENVIRONMENT="us-east-1"
GOOGLE_API_KEY=GOOGLE_API_KEY

In [83]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings
#create embedding means numeric representation of text provided or document
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # lates model for embedding from google
em=embeddings.embed_query("H")
em[:3]


[0.0237551461905241, -0.060975316911935806, -0.0640849843621254]

In [84]:
from langchain_community.vectorstores import VectorStore

In [85]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

In [89]:
index_name = "rag-index" # name must be lower case and ''
pc = Pinecone(api_key=api_key)

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)
print(index)
vector_store = PineconeVectorStore(embedding=embeddings, index=index) #Vector Store

In [90]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
# vectors=[]
for data in tqdm(dataset):
    vector = embeddings.embed_query(data.page_content) #create embedding numeric

    # Generate a unique ID for each document (e.g., UUID or hash)
    data_id = str(hash(data.page_content)) #uuid or Hash doc_id = 22i3y12jdyuew****

    # Create metadata dictionary
    data_metadata = {"source": data.metadata["source"]}
    data_list=[data]
    # Upsert the vector with a unique ID and metadata
    # index.upsert(vectors=[(doc_id, vector, doc_metadata)]) # Include metadata in upsert
    vector_store.add_documents(documents=data_list, id=data_id)

  8%|▊         | 82/1000 [00:38<07:09,  2.14it/s]


GoogleGenerativeAIError: Error embedding content: 429 Quota exceeded for quota metric 'Batch Embed Content API requests' and limit 'Batch embed contents request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:502430975926'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/batch_embed_contents_requests"
}
metadata {
  key: "quota_location"
  value: "us-east7"
}
metadata {
  key: "quota_limit"
  value: "BatchEmbedContentsRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "150"
}
metadata {
  key: "consumer"
  value: "projects/502430975926"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quotas/help/request_increase"
}
]

In [110]:
retriver = vector_store.as_retriever(search_kwargs={"k": 1})
#print(retriver)


In [117]:
from pprint import pprint
user_quory:str = "show  five History question"
docs = retriver.get_relevant_documents(user_quory)
pprint(docs)

[Document(id='ec0ace05-b161-4d20-95cb-01a0cca4e7ef', metadata={'row': 52.0, 'source': '/content/educational_quiz_generator.csv'}, page_content='Topic: History\nQuestion: Question 53 on History')]


In [118]:
prompt_template = f"""Give answer of {user_quory} from the rag context {docs} """
response = model_2_0.invoke(prompt_template)

In [119]:
pprint(response.content)

('Okay, here are five history questions based on the provided context and '
 'assuming a general level of history knowledge:\n'
 '\n'
 '**Please note:** The provided context is very minimal. It only indicates a '
 '"History" topic and a question number. Therefore, I will create general '
 'history questions.\n'
 '\n'
 'Here are five questions:\n'
 '\n'
 '1.  **Question 1:**  What were the major causes of World War I? (This is a '
 'broad question that could lead to discussions about nationalism, '
 'imperialism, militarism, and alliances)\n'
 '\n'
 '2.  **Question 2:**  Who was the first President of the United States, and '
 'what were some of the major challenges he faced during his presidency? (This '
 'focuses on a key figure and early American history)\n'
 '\n'
 '3.  **Question 3:**  Describe the significance of the Silk Road in world '
 'history. (This tests knowledge of trade routes and cultural exchange.)\n'
 '\n'
 '4.  **Question 4:** What was the impact of the Industrial Revo